## 1. 提取 csv 的第一列的 taxonomy 信息

- 提取了 n 个 taxonomic 信息，则后续就会在 SILVA 数据库中查询这 n 个 taxonomic 信息是否存在


In [ ]:
import pandas as pd

PATH="/Users/kiancai/STA24/CWD/STAi/MiCoGPT/data/try2_withCC/abundance_all_90338.csv"

df = pd.read_csv(PATH)
first_column = df.iloc[:, 0]

# 保存为 full_taxonomy.csv
first_column.to_csv("full_taxonomy.csv", index=False)
print(first_column[:5])


0    k__Bacteria;p__Proteobacteria;c__Gammaproteoba...
1    k__Bacteria;p__Bdellovibrionota;c__Bdellovibri...
2    k__Bacteria;p__Proteobacteria;c__Gammaproteoba...
3    k__Bacteria;p__Firmicutes;c__Clostridia;o__Eub...
4    k__Bacteria;p__Proteobacteria;c__Gammaproteoba...
Name: Taxonomy, dtype: object


## 2. check match 的情况

- 检查 SILVA 数据库中是否有对应的 taxonomic 信息，以及每个 taxonomic 信息查询到的对应的 SILVA 数据库中代表序列的数量

In [3]:
import pandas as pd

# 物种情况
csv_path = "full_taxonomy.csv"
tax_df = pd.read_csv(csv_path)
tax_df.columns = ["Taxonomy"]

# silva 的物种情况
silva_tax_path = "data/silva-138-99-tax-exported/taxonomy.tsv"
silva_df = pd.read_csv(silva_tax_path, sep="\t")

# 匹配 genus
tax_df["Genus"] = tax_df["Taxonomy"].str.extract(r"(g__[^;]+)")
silva_df["Genus"] = silva_df["Taxon"].str.extract(r"(g__[^;]+)")

# 丢弃没有匹配到的
query_genera = tax_df["Genus"].dropna().unique()
silva_sub = silva_df[silva_df["Genus"].isin(query_genera)].copy()

genus_counts = (
    silva_sub
    .groupby("Genus")
    .size()
    .sort_values(ascending=False)
)

print("每个 genus 在 SILVA 中匹配到的行数：")
display(genus_counts)

每个 genus 在 SILVA 中匹配到的行数：


Genus
g__Bacillus             12014
g__Pseudomonas           7070
g__Streptomyces          5809
g__Streptococcus         4635
g__Staphylococcus        4443
                        ...  
g__Peptoanaerobacter        2
g__Gulbenkiania             2
g__Faucicola                1
g__Jonquetella              1
g__Oceanotoga               1
Length: 1117, dtype: int64

## 3. Tokenization

- 一次性对匹配到的所有序列进行 tokenization

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

import numpy as np
import pandas as pd
from tqdm import tqdm
from Bio import SeqIO
from transformers import AutoTokenizer, AutoModel
from concurrent.futures import ThreadPoolExecutor, as_completed


# 1. 加载 tokenizer 和 model（CPU），获取真实 max_len

tokenizer = AutoTokenizer.from_pretrained(
    "zhihan1996/DNABERT-S",
    trust_remote_code=True
)

model = AutoModel.from_pretrained(
    "zhihan1996/DNABERT-S",
    trust_remote_code=True
)

max_len = model.config.max_position_embeddings
print("模型真正的最大 token 长度（max_position_embeddings）：", max_len)


# 2. 读取 SILVA 子集和 FASTA
N = silva_sub.shape[0]
print("需要 embedding 的总序列数量:", N)

fasta_path = "data/silva-138-99-seqs-exported/dna-sequences.fasta"
seq_dict = {r.id: str(r.seq) for r in SeqIO.parse(fasta_path, "fasta")}
print("SILVA 中包含的 FASTA 总量:", len(seq_dict))



# 3. 单条序列的 tokenization
def tokenize_one(args):
    idx, fid, genus = args
    seq = seq_dict[fid]

    tokens = tokenizer(
        seq,
        return_tensors="np",
        truncation=False,
        padding=False
    )["input_ids"][0]   # shape=(L,)

    token_len = len(tokens)
    truncated = token_len > max_len

    return idx, fid, genus, tokens, token_len, truncated



# 4. 多线程 tokenizer（32 线程可改）
THREADS = 32

# 注意：这里我们按 idx 存，保证和 silva_sub 一一对应
token_arrays = [None] * N
token_lengths = np.zeros(N, dtype=np.int32)
feature_ids = np.empty(N, dtype=object)
genera = np.empty(N, dtype=object)
truncated_flags = np.zeros(N, dtype=np.uint8)

jobs = [(i, row["Feature ID"], row["Genus"]) for i, (_, row) in enumerate(silva_sub.iterrows())]

with ThreadPoolExecutor(max_workers=THREADS) as executor:
    futures = [executor.submit(tokenize_one, j) for j in jobs]

    for f in tqdm(as_completed(futures), total=N, desc="Tokenizing", ncols=100):
        idx, fid, genus, tokens, token_len, truncated = f.result()

        token_arrays[idx] = tokens
        token_lengths[idx] = token_len
        feature_ids[idx] = fid
        genera[idx] = genus
        truncated_flags[idx] = truncated

print("Tokenization 完成！")
print("被截断的序列数量：", int(truncated_flags.sum()))

# 统计最长的 token 序列长度
max_token_len = int(token_lengths.max())
print("所有序列中最长的 token 长度：", max_token_len)

# 你如果想顺便看一下长度分布，也可以加一个简单统计：
# print("token length 分位数：", np.percentile(token_lengths, [50, 90, 95, 99]))


# 5. 保存结果（NPZ）
np.savez_compressed(
    "tokens.npz",
    token_arrays=np.array(token_arrays, dtype=object),
    token_lengths=token_lengths,
    feature_ids=feature_ids,
    genera=genera,
    truncated=truncated_flags
)

print("Token 文件保存到 tokens.npz")


Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
/home/cml_lab/anaconda3/envs/caiqy_DNABERT_S/lib/python3.9/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
/home/cml_lab/caiqy/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-S/00e47f96cdea35e4b6f5df89e5419cbe47d490c6/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(


模型真正的最大 token 长度（max_position_embeddings）： 512
tokenizer.model_max_length（仅供参考，可能是占位值）： 1000000000000000019884624838656
总序列数量: 264926
FASTA 加载数量: 436680


Tokenizing: 100%|███████████████████████████████████████| 264926/264926 [00:01<00:00, 146082.61it/s]


Tokenization 完成！
被截断的序列数量： 23
所有序列中最长的 token 长度： 732
Token 文件保存到 tokens.npz


## 4. filter 

- 由于上一步，部分 taxonomic 信息在 SILVA 数据库中的全长序列进行 tokenization 后，得到的 tokenized 序列长度超过了 DNABERT-S 的最大输入长度（512），因此需要对这些序列进行过滤。又由于 DNABERT-S 采用了 BPE，所以只有在进行了 tokenized 以后我们才可能知道到底哪些序列长度会超出 512。因此，我们需要对 tokenized 后的序列进行过滤，只保留长度小于等于 512 的序列。

In [9]:
import numpy as np
import pandas as pd

# 1. 读取 tokenizer 结果
data = np.load("tokens.npz", allow_pickle=True)

token_arrays = data["token_arrays"]      # dtype=object, 每条是不定长 token 序列
token_lengths = data["token_lengths"]    # 每条的 token 长度
feature_ids = data["feature_ids"]        # 和 silva_sub["Feature ID"] 对应
genera = data["genera"]                  # 属
truncated_flags = data["truncated"]      # 0/1

N = len(token_arrays)
print("总序列数:", N)
print("被标记为截断的序列数:", int(truncated_flags.sum()))

# 2. 构造一个 DataFrame，方便分析被截断的序列
df = pd.DataFrame({
    "idx": np.arange(N),
    "FeatureID": feature_ids,
    "Genus": genera,
    "TokenLength": token_lengths,
    "Truncated": truncated_flags.astype(bool),
})

trunc_df = df[df["Truncated"]].copy()
print("\n=== 所有被截断的序列（共 {} 条） ===".format(len(trunc_df)))
display(trunc_df[["idx", "Genus", "FeatureID", "TokenLength"]])

# 3. 每个 Genus 的总序列数（基于当前 26w 条）
genus_total_counts = df["Genus"].value_counts()
print("\n=== 每个 Genus 的总序列数（前几行预览） ===")
display(genus_total_counts.head())

# 4. 决定要删除哪些被截断的序列
# 规则：
#   - 对于每个 genus：
#       * 如果 genus_total_counts[genus] > 截断条数：可以删除该 genus 所有被截断的序列
#       * 如果 genus_total_counts[genus] == 截断条数：说明这个 genus 只有这些长序列，
#         为了“不让这个 genus 完全消失”，至少保留其中 1 条（保留 token 最短的那条）

to_drop_idx = []

grouped = trunc_df.groupby("Genus")

for genus, group in grouped:
    total = genus_total_counts.get(genus, 0)
    num_trunc = group.shape[0]

    if total > num_trunc:
        # 该 genus 还有别的（未截断）代表性序列，所有被截断的都可以安全删除
        to_drop_idx.extend(group["idx"].tolist())
    else:
        # total == num_trunc：所有代表性序列都太长
        # 至少保留其中 1 条，这里选择 token 最短的一条保留，其余删除
        group_sorted = group.sort_values("TokenLength")
        keep_idx = int(group_sorted.iloc[0]["idx"])
        drop_idxs = [int(i) for i in group_sorted["idx"].tolist() if int(i) != keep_idx]
        to_drop_idx.extend(drop_idxs)

to_drop_idx = sorted(set(to_drop_idx))

print("\n=== 删除计划统计 ===")
print("被截断序列总数:           ", len(trunc_df))
print("计划删除的截断序列条数:   ", len(to_drop_idx))
print("计划保留但仍被截断的条数: ", len(trunc_df) - len(to_drop_idx))

# 5. 看看这些被截断序列分别来自哪些 Genus 以及每个 Genus 的删除情况
trunc_df["WillDrop"] = trunc_df["idx"].isin(to_drop_idx)

genus_trunc_summary = (
    trunc_df
    .groupby("Genus")
    .agg(
        TotalInGenus=("Genus", lambda x: genus_total_counts.loc[x.name]),
        TruncCount=("Genus", "size"),
        DropCount=("WillDrop", "sum"),
    )
    .sort_values("TruncCount", ascending=False)
)

print("\n=== 各 Genus 的截断情况与删除计划 ===")
display(genus_trunc_summary)

# 6. 实际执行过滤，构造保留的 mask
keep_mask = np.ones(N, dtype=bool)
keep_mask[to_drop_idx] = False

print("\n过滤后总序列数:", int(keep_mask.sum()))

# 再检查一遍：过滤之后，每个 genus 是否至少保留 1 条
df_filtered = df[keep_mask].copy()
genus_filtered_counts = df_filtered["Genus"].value_counts()

missing_genus = genus_total_counts.index[~genus_total_counts.index.isin(genus_filtered_counts.index)]
print("过滤后完全消失的 genus 数量:", len(missing_genus))

if len(missing_genus) > 0:
    print("⚠ 以下 genus 在过滤后完全没有代表性序列（理论上不应该发生）：")
    display(missing_genus)
else:
    print("✅ 过滤后，每个 genus 至少保留 1 条代表性序列。")

# 7. 保存过滤后的 token 结果，供后续 embedding 使用
token_arrays_filtered = token_arrays[keep_mask]
token_lengths_filtered = token_lengths[keep_mask]
feature_ids_filtered = feature_ids[keep_mask]
genera_filtered = genera[keep_mask]
truncated_filtered = truncated_flags[keep_mask]

np.savez_compressed(
    "tokens_filtered.npz",
    token_arrays=np.array(token_arrays_filtered, dtype=object),
    token_lengths=token_lengths_filtered,
    feature_ids=feature_ids_filtered,
    genera=genera_filtered,
    truncated=truncated_filtered
)

print("\n✅ 已保存过滤后的 tokens 到 tokens_filtered.npz")


总序列数: 264926
被标记为截断的序列数: 23

=== 所有被截断的序列（共 23 条） ===


,idx,Genus,FeatureID,TokenLength
37545,37545,g__Streptococcus,CAQA01000070.1409.4877,732
37844,37844,g__Streptococcus,CIIB01000023.4139.7040,615
37847,37847,g__Streptococcus,CKGV01000033.4234.7508,663
38293,38293,g__Mycoplasma,CP001047.191404.193931,523
38616,38616,g__Thermus,CP001962.592399.595224,617
39053,39053,g__Corynebacterium,CP002857.115758.118576,609
40387,40387,g__Bacillus,CP008712.321643.324759,631
41291,41291,g__Bacillus,CP010106.282981.285881,596
42084,42084,g__Staphylococcus,CP012409.2263835.2266601,573
49659,49659,g__Streptococcus,CRPA01000007.15140.18272,660



=== 每个 Genus 的总序列数（前几行预览） ===


Genus
g__Bacillus          12014
g__Pseudomonas        7070
g__Streptomyces       5809
g__Streptococcus      4635
g__Staphylococcus     4443
Name: count, dtype: int64


=== 删除计划统计 ===
被截断序列总数:            23
计划删除的截断序列条数:    23
计划保留但仍被截断的条数:  0

=== 各 Genus 的截断情况与删除计划 ===


/home/cml_lab/tmp/ipykernel_19680/2743670463.py:72: FutureWarning: Pinning the groupby key to each group in SeriesGroupBy.agg is deprecated, and cases that relied on it will raise in a future version. If your operation requires utilizing the groupby keys, iterate over the groupby object instead.
  trunc_df


,TotalInGenus,TruncCount,DropCount
Genus,,,
g__Streptococcus,4635,5,5
g__Staphylococcus,4443,3,3
g__Bacillus,12014,2,2
g__Candidatus_Adlerbacteria,34,1,1
g__Methanobrevibacter,961,1,1
g__Micrococcus,295,1,1
g__Candidatus_Jorgensenbacteria,10,1,1
g__Corynebacterium,2945,1,1
g__Mycoplasma,573,1,1



过滤后总序列数: 264903
过滤后完全消失的 genus 数量: 0
✅ 过滤后，每个 genus 至少保留 1 条代表性序列。

✅ 已保存过滤后的 tokens 到 tokens_filtered.npz


## 5. embedding

- 对上一步 filtered 后的序列进行 embedding，得到每个序列的 embedding 向量

In [ ]:
import torch
import numpy as np
import h5py
from tqdm import tqdm
from transformers import AutoModel


# 1. GPU & 模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("使用设备：", device)

model = AutoModel.from_pretrained(
    "zhihan1996/DNABERT-S",
    trust_remote_code=True
).to(device).eval()



# 2. 加载 token 文件，使用过滤后的 tokens_filtered.npz
data = np.load("tokens_filtered.npz", allow_pickle=True)
token_arrays = data["token_arrays"]      # object array: 每个元素是一条 token 序列（np.array）
token_lengths = data["token_lengths"]
feature_ids = data["feature_ids"]
genera = data["genera"]
truncated_flags = data["truncated"]

N = len(token_arrays)
print("加载 Token 数量:", N)
print("其中标记为 Truncated 的条数:", int(truncated_flags.sum()))


# 3. 建立 HDF5 文件
h5_path = "embeddings.h5"
h5f = h5py.File(h5_path, "w")

emb_ds = h5f.create_dataset("embeddings", (N, 768), dtype="float32", compression="gzip")
fid_ds = h5f.create_dataset("feature_ids", (N,), dtype=h5py.string_dtype('utf-8'))
genus_ds = h5f.create_dataset("genus", (N,), dtype=h5py.string_dtype('utf-8'))
trunc_ds = h5f.create_dataset("truncated", (N,), dtype="uint8")



# 4. GPU batch embedding（不再 tokenizer）
BATCH_SIZE = 128      # A100 上 128 或 256 都可以

@torch.no_grad()
def embed_batch(token_batch):
    # token_batch: list/array of 1D numpy arrays, 每个是一个 token 序列
    max_len = max(len(t) for t in token_batch)
    padded = np.zeros((len(token_batch), max_len), dtype=np.int64)
    for i, t in enumerate(token_batch):
        padded[i, :len(t)] = t

    tokens = torch.tensor(padded, dtype=torch.long).to(device)
    hidden = model(tokens)[0]         # (B, L, 768)
    emb = hidden.mean(dim=1)          # (B, 768)
    return emb.cpu().numpy().astype(np.float32)



# 5. 主循环（GPU 纯 forward）
progress = tqdm(range(0, N, BATCH_SIZE), desc="Embedding", dynamic_ncols=True)

for start in progress:
    end = min(start + BATCH_SIZE, N)

    batch_tokens = token_arrays[start:end]
    batch_emb = embed_batch(batch_tokens)

    emb_ds[start:end] = batch_emb
    fid_ds[start:end] = feature_ids[start:end]
    genus_ds[start:end] = genera[start:end]
    trunc_ds[start:end] = truncated_flags[start:end]

h5f.close()

print("Embedding 完成！写入 embeddings.h5")


使用设备： cuda


/home/cml_lab/anaconda3/envs/caiqy_DNABERT_S/lib/python3.9/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
/home/cml_lab/anaconda3/envs/caiqy_DNABERT_S/lib/python3.9/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
/home/cml_lab/caiqy/.cache/huggingface/modules/trans

加载 Token 数量: 264903
其中标记为 Truncated 的条数: 0


Embedding: 100%|████████████████████████████████████████████| 4140/4140 [38:09<00:00,  1.81it/s]


Embedding 完成！写入 embeddings.h5


## 6. mean pooling

- 对上一步得到的每个序列的 embedding 向量进行 mean pooling，得到每个 taxonomic 信息的 embedding 向量

In [11]:
import numpy as np
import pandas as pd
import h5py
from tqdm import tqdm


# 1. 读取 HDF5 中的 embedding 和 Genus
h5_path = "embeddings.h5"

with h5py.File(h5_path, "r") as f:
    embeddings = f["embeddings"][...]   # shape: (N, 768)
    genera = f["genus"][...]            # h5py string dtype -> numpy array
    # 如果类型是 bytes，转成字符串
    genera = np.array(genera, dtype=str)

N, D = embeddings.shape
print(f"总 embedding 数量: {N}, 维度: {D}")
print(f"不同 genus 数量: {len(np.unique(genera))}")


# 2. 按 genus 分组，计算平均 embedding
# 用 DataFrame 只是为了方便 groupby，不把 embeddings 放进去，避免太大
df = pd.DataFrame({
    "Genus": genera,
    "Idx": np.arange(N)
})

groups = df.groupby("Genus")["Idx"].apply(list)

genus_list = []
genus_emb_list = []
genus_counts = []

print("开始按 genus 聚合 embedding ...")

for genus, idx_list in tqdm(groups.items(), total=len(groups)):
    idx_array = np.array(idx_list, dtype=int)
    genus_emb = embeddings[idx_array].mean(axis=0)  # (768,)
    
    genus_list.append(genus)
    genus_emb_list.append(genus_emb)
    genus_counts.append(len(idx_array))

genus_embeddings = np.vstack(genus_emb_list)  # shape: (G, 768)
genus_counts = np.array(genus_counts, dtype=np.int32)

print("聚合完成！")
print("最终 genus 数量:", genus_embeddings.shape[0])


# 3. 保存 genus-level embedding（npz）
np.savez_compressed(
    "genus_embeddings.npz",
    genus=np.array(genus_list, dtype=object),
    embeddings=genus_embeddings,
    counts=genus_counts
)

print("✅ genus-level embedding 已保存到 genus_embeddings.npz")
print("  - embeddings 形状:", genus_embeddings.shape)


# 4. 输出每个 genus 对应的 embedding 数量统计 CSV
genus_stats_df = pd.DataFrame({
    "Genus": genus_list,
    "NumEmbeddings": genus_counts
}).sort_values("NumEmbeddings", ascending=False)

genus_stats_df.to_csv("genus_embedding_counts.csv", index=False)

print("✅ 每个 genus 对应的 embedding 数量统计已保存到 genus_embedding_counts.csv")
print("前几行预览：")
display(genus_stats_df.head())


总 embedding 数量: 264903, 维度: 768
不同 genus 数量: 1117
开始按 genus 聚合 embedding ...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1117/1117 [00:00<00:00, 5266.51it/s]


聚合完成！
最终 genus 数量: 1117
✅ genus-level embedding 已保存到 genus_embeddings.npz
  - embeddings 形状: (1117, 768)
✅ 每个 genus 对应的 embedding 数量统计已保存到 genus_embedding_counts.csv
前几行预览：


,Genus,NumEmbeddings
142,g__Bacillus,12012
816,g__Pseudomonas,7070
961,g__Streptomyces,5809
960,g__Streptococcus,4630
954,g__Staphylococcus,4440


## 7.check

- 简单查看一下最后输出的 genus_embeddings.npz 文件，看看是否符合预期

In [3]:
import numpy as np
import pandas as pd

data = np.load("genus_embeddings_256.npz", allow_pickle=True)

print("包含的数组名称:", data.files)

genus = data["genus"]
embeddings = data["embeddings"]
counts = data["counts"]

print("genus 形状:", genus.shape)
print("embeddings 形状:", embeddings.shape)
print("counts 形状:", counts.shape)

print("\n前 5 个 genus:")
print(genus[:5])

print("\n前 5 个 genus 的 counts:")
print(counts[:5])

print("\n第 1 个 genus 的 embedding 向量前 10 维:")
print(embeddings[0][:10])


包含的数组名称: ['genus', 'embeddings', 'counts', 'explained_variance_ratio']
genus 形状: (1117,)
embeddings 形状: (1117, 256)
counts 形状: (1117,)

前 5 个 genus:
['g__0319-6G20' 'g__0319-7L14' 'g__11-24' 'g__1174-901-12' 'g__28-YEA-48']

前 5 个 genus 的 counts:
[362  35 144  61   2]

第 1 个 genus 的 embedding 向量前 10 维:
[ 0.29121813  0.28034773  0.03950644 -0.14446126 -0.14121476  0.09987964
 -0.08377898 -0.01667265 -0.00424662 -0.00804974]
